# Video Processing for BT Lab
This notebook extracts metadata from video files 

<br>

![UofC logo](./pictures/uofc_logo-black.jpg)

In [15]:
#import libraries
import os
import cv2
import json
import json
import datetime
import pandas as pd
import numpy as np

In [2]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [3]:
def get_video_properties(mp4_file: str) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        old_fps = video.get(cv2.CAP_PROP_FPS)
        number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
        length = number_of_frames / old_fps
        video_time = datetime.timedelta(seconds=length)
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video_properties.append(video_time)
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    
    return video_properties

In [4]:
def add_video_properties(metadata:dict) -> dict:
    new_patient_info = {}

    for patient_id, patient_info in metadata.items():
        try:
            video_path = patient_info["local path"]
            frames, fps, length_in_seconds, length_in_datetime = get_video_properties(video_path)
            patient_info["frames"] = frames
            patient_info["old fps"] = fps
            patient_info["length"] = length_in_seconds
            patient_info["length (datetime)"] = length_in_datetime
            new_patient_info[patient_id] = patient_info
        except Exception as e:
            print(f"{type(e)} - {patient_info['ID']}")

    return new_patient_info 

In [5]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [6]:
def export_to_excel(patient_info:dict, filename: str):
    patient_df = pd.DataFrame.from_dict(patient_info).transpose()
    patient_df = patient_df.drop(columns=["local path"])
    new_df = patient_df.set_index(["ID", "distance", "breathing", "blanket", "filename"])
    new_df.to_excel(filename)
    return new_df

In [ ]:
def print_aggregate_stats(patient_df):
    patient_df

## Get video metadata and export to xlsx file (stage 2.1)

In [7]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb.json")
metadata_thermal = load_json(json_dir, "thermal.json")

In [8]:
""" extract metadata from video (rgb) """

complete_metadata_rgb = add_video_properties(metadata_rgb)

In [9]:
""" patient info to pandas DataFrame (rgb)"""
rgb_xlsx = repo_dir + "/records/xlsx/rgb.xlsx"
rgb_df = export_to_excel(complete_metadata_rgb, rgb_xlsx)

In [10]:
""" extract metadata from video (thermal) """

complete_metadata_thermal = add_video_properties(metadata_thermal)

In [11]:
""" patient info to pandas DataFrame (thermal)"""
thermal_xlsx = repo_dir + "/records/xlsx/thermal.xlsx"
thermal_df = export_to_excel(complete_metadata_thermal, thermal_xlsx)